In [9]:
from langgraph.graph import END, START, StateGraph
from langchain_ollama import ChatOllama
from typing import TypedDict , Annotated , Literal
from pydantic import BaseModel,Field
import operator

state

In [2]:
class QuadState(TypedDict):
    a: int
    b: int
    c: int

    equation: str
    discriminant: float
    result: str

In [10]:
def show_equation(state: QuadState):
    equation = f"{state['a']}x^2 + {state['b']}x + {state['c']} = 0"
    return { 
        **state,
        "equation": equation
    }

In [11]:
def calculate_discriminant(state: QuadState):
    discriminant = state["b"]**2 - (4*state["a"]*state["c"])
    return { 
        **state,
        "discriminant": discriminant
    }

In [12]:
def real_roots(state:QuadState):
    root1 = (-state["b"] + math.sqrt(state["discriminant"])) / (2 * state["a"])
    root2 = (-state["b"] - math.sqrt(state["discriminant"])) / (2 * state["a"])
    result = f"Roots are {root1} and {root2}"
    return {"result": result}

In [13]:
def repeated_roots(state:QuadState):
    root = -state["b"] / (2 * state["a"])
    result = f"Repeating Root is {root}"
    return {"result": result}


In [14]:
def no_real_roots(state:QuadState):
    result = "No real roots"
    return {"result": result}

In [15]:
def check_condition(state:QuadState) -> Literal["real_roots", "no_real_roots", "repeated_roots"]:
    if state["discriminant"] > 0:
        return "real_roots"
    elif state["discriminant"] == 0:
        return "repeated_roots"
    else:
        return "no_real_roots"

graph

In [17]:
# Build Graph
graph = StateGraph(QuadState)

graph.add_node("show_equation", show_equation)
graph.add_node("calculate_discriminant", calculate_discriminant)
graph.add_node("real_roots", real_roots)
graph.add_node("repeated_roots", repeated_roots)
graph.add_node("no_real_roots", no_real_roots)

graph.add_edge(START, "show_equation")
graph.add_edge("show_equation", "calculate_discriminant")

graph.add_conditional_edges(
    "calculate_discriminant",
    check_condition,
)

graph.add_edge("real_roots", END)
graph.add_edge("repeated_roots", END)
graph.add_edge("no_real_roots", END)

workflow = graph.compile()

In [18]:
initial_state = {
    "a": 4,
    "b": -5,
    "c": 6,
}

In [19]:
final_state = workflow.invoke(initial_state)
print(final_state)

{'a': 4, 'b': -5, 'c': 6, 'equation': '4x^2 + -5x + 6 = 0', 'discriminant': -71, 'result': 'No real roots'}
